In [ ]:
# good code
# chaining

def tweak_autos(autos):
    cols = ["city08", "comb08", "highway08", "cylinders", "displ", "drive", "eng_dscr", 
            "fuelCost08", "make", "model", "trany", "range", "createdOn", "year"]
    return (autos
            [cols]
            .assign(cylinders=autos.cylinders.fillna(0).astype("int8"),
                    displ=autos.displ.fillna(0).astype("float16"),
                    drive=autos.drive.fillna("Other").astype("category"),
                    automatic=autos.trany.str.contains("Auto"),
                    speeds=autos.trany.str.extract(r"(\d)+").fillna("20").astype("int8"),
                    createdOn=pd.to_datetime(autos.createdOn.replace({" EDT": "-04:00",
                                                                      " EST": "-05:00"}, regex=True)),
                    ffs=autos.eng_dscr.str.contains("FFS")
                   )
            .astype({"highway08": "int8", "city08": "int16", "comb08": "int16", "fuelCost": "int16",
                     "range": "int16", "year": "int16", "make": "category"})
            .drop(columns=["trany", "eng_dscr"])
           )

In [ ]:
def country(val):
    if val in {"Chevrolet", "Ford", "Dodge", "GMC", "Tesla"}:
        return "US"
    return "Other"

In [ ]:
%%timeit
(autos2
 .assign(country=autos2.make.apply(country))
)

In [ ]:
%%timeit
values = {"Chevrolet", "Ford", "Dodge", "GMC", "Tesla"}
(autos2
 .assign(country="US")
 .assign(country=lambda df_:df_country.where(df_.make.isin(values), "Other"))
)

In [ ]:
%%timeit
(autos2
 .assign(country=np.select([autos2.make.isin({"Chevrolet", "Ford", "Dodge", "GMC", "Tesla"})],
                           ["US"], "Other"))

In [ ]:
(autos2
 .assign(country=autos2.make.apply(country))
 .groupby(["year", "country"])
 .mean()
 .unstack()
 .city08
 .rolling(2)
 .mean().plot()
 .legend(bbox_to_anchor=(1,1))
)

In [ ]:
def vals_gt(df_, num):
    return df_[df_.gt(num)].dropna()

(autos2
 .assign(country=autos2.make.apply(country))
 .groupby(["year", "country"])
 .count()
 .pipe(vals_gt, 700)
)